<a href="https://colab.research.google.com/github/naritaku/Googlecolaboratory/blob/master/kaiketugiho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

スケジューリング問題を巡回セールスマン問題に置き換えて色々してみようとした時の断片

In [0]:
# coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance as dis
from mpl_toolkits.mplot3d import Axes3D
import csv
from sklearn.cluster import KMeans

glapfcoilRoute = ["sort0.csv", "sort1.csv", "sort2.csv"]


def inputData(filename):
    d = np.loadtxt(filename, delimiter=",")
    return d
# 距離テーブルを作成


def makeSeikiTable():
    d = np.random.normal(0.5, 0.15, (90, 3))
    size = len(d)
    table = np.ones((size, size)) * -1
    # 幅:w 厚み:t 目付量:a
    dataMax = [1.0, 1.0, 1.0]
    dataMin = [0.0, 0.0, 0.0]
    # 制約条件 130%~70%まで許容
    # P1, Q1, R
    big = [0.3, 0.3, 0.3]
    # P2, Q2, R
    small = [0.3, 0.3, 0.3]
    costRate = [10.0, 10.0, 10.0]
    for i in xrange(size):
        for k in xrange(3):
            d[i][k] = (d[i][k] - dataMin[k]) / (dataMax[k] - dataMin[k])
    for i in xrange(size):
        for j in xrange(size):
            if i == j:
                table[i][j] = 3e100
            else:
                diff = 0.0
                cost = 0.0
                for k in xrange(3):
                    diff = abs(d[j][k] - d[i][k])
                    if d[i][k] > d[j][k] and diff > small[k]:
                        cost = 1e50
                        break
                    elif d[i][k] < d[j][k] and diff > big[k]:
                        cost = 1e50
                        break
                    else:
                        funccost = abs(diff) * costRate[k]
                        cost = cost + funccost
                table[i][j] = cost
    return table

# 距離テーブルを作成


def makeTable(d):
    size = len(d)
    table = np.ones((size, size)) * -1
    # 幅:w 厚み:t 目付量:a
    dataMax = [1800.0, 3.2, 0.040]
    dataMin = [600.0, 0.4, 0.007]
    # 制約条件 130%~70%まで許容
    # P1, Q1, R
    big = [0.3, 0.3, 0.3]
    # P2, Q2, R
    small = [0.3, 0.3, 0.3]
    costRate = [10.0, 10.0, 10.0]
    for i in xrange(size):
        for k in xrange(3):
            d[i][k] = (d[i][k] - dataMin[k]) / (dataMax[k] - dataMin[k])
    for i in xrange(size):
        for j in xrange(size):
            if i == j:
                table[i][j] = 3e100
            else:
                diff = 0.0
                cost = 0.0
                for k in xrange(3):
                    diff = abs(d[j][k] - d[i][k])
                    if d[i][k] > d[j][k] and diff > small[k]:
                        cost = 1e50
                        break
                    elif d[i][k] < d[j][k] and diff > big[k]:
                        cost = 1e50
                        break
                    else:
                        funccost = abs(diff) * costRate[k]
                        cost = cost + funccost
                table[i][j] = cost
    return table


def cost(costTable, route):
    cost = 0
    for i in range(len(route) - 1):
        cost += costTable[route[i]][route[i + 1]]
    return float(cost)

# 一度数珠繋ぎにして最大のコストの部分を始点と終点となるようにする
# 局所解の時、始点と終点のコイルが決まっていなければ解を有効
# (結局これに頼ってるようでは最適化できてないし、これで求まった解も局所解でしかない)


def change_startCoil(coilList, costTable):
    #print "before:",
    #print coilList

    route = []
    maxcost = costTable[int(coilList[0])][int(coilList[1])]
    start = 0
    for i in xrange(len(coilList)):
        next = int(coilList[i])
        prev = int(coilList[i - 1])
        if maxcost < costTable[prev][next]:
            maxcost = costTable[prev][next]
            start = i
    for i in xrange(start, len(coilList)):
        route.append(coilList[i])
    for i in xrange(0, start):
        route.append(coilList[i])
    #print "After:",
    #print route
    return route

# Nearest Neighbour法 (貪欲法)
# 次に近いものを今までのルートの中で一番コストが低くなる位置に再配置


def NN(costTable):
    coilNum = len(costTable)
    willCoil = []
    for i in range(coilNum):
        willCoil.append(i)
    nowCoil = int(np.random.random() * coilNum % coilNum)
    willCoil.remove(nowCoil)
    nextCoil = 0
    coilRoute = []
    cost = 0
    for i in range(coilNum - 2):
        coilRoute.append(nowCoil)
        minCost = 1e101
        for j in willCoil:
            if costTable[nowCoil][j] < minCost:
                nextCoil = j
                minCost = costTable[nowCoil][nextCoil]
        nowCoil = nextCoil
        cost += minCost
        willCoil.remove(nextCoil)
    coilRoute.append(nowCoil)
    coilRoute.append(willCoil[0])
    coilRoute = change_startCoil(coilRoute, costTable)
    #print "cost:",
    #print cost
    #print coilRoute
    return coilRoute


# 2opt()
def opt2(costTable, coilRoute):
    size = len(coilRoute)
    total = 0
    while True:
        count = 0
        # coilRoute=change_startCoil(coilRoute,costTable)
        for i in xrange(size - 2):
            i1 = i + 1
            for j in xrange(i + 2, size - 1):
                if j == size - 1:
                    j1 = 0
                else:
                    j1 = j + 1
                if i != 0 or j1 != 0:
                    l1 = costTable[coilRoute[i]][coilRoute[i1]]
                    l2 = costTable[coilRoute[j]][coilRoute[j1]]
                    l3 = costTable[coilRoute[i]][coilRoute[j]]
                    l4 = costTable[coilRoute[i1]][coilRoute[j1]]
                    if l1 + l2 > l3 + l4:
                        new_coilRoute = coilRoute[i1:j + 1]
                        coilRoute[i1:j + 1] = new_coilRoute[::-1]
                        count += 1
        total += count
        if count == 0:
            break
#	print "コスト",
#	print cost(costTable,coilRoute)
#	print coilRoute
    return coilRoute

# 2opt+焼きなまし


def sa2opt(costTable, coilRoute):
    temp = 10
    size = len(coilRoute)
    total = 0
    while temp > 1e-6:
        count = 0
        for i in xrange(size - 2):
            i1 = i + 1
            for j in xrange(i + 2, size):
                if j == size - 1:
                    j1 = 0
                else:
                    j1 = j + 1
                if i != 0 or j1 != 0:
                    l1 = costTable[coilRoute[i]][coilRoute[i1]]
                    l2 = costTable[coilRoute[j]][coilRoute[j1]]
                    l3 = costTable[coilRoute[i]][coilRoute[j]]
                    l4 = costTable[coilRoute[i1]][coilRoute[j1]]
                    if l1 + l2 > l3 + l4:
                        new_coilRoute = coilRoute[i1:j + 1]
                        coilRoute[i1:j + 1] = new_coilRoute[::-1]
                        count += 1
                    else:
                        dE = l1 + l2 - (l3 + l4)
                        temp *= 0.99995
                        if np.exp(dE / temp) > np.random.random():
                            new_coilRoute = coilRoute[i1:j + 1]
                            coilRoute[i1:j + 1] = new_coilRoute[::-1]
        total += count
        if count == 0:
            break
    coilRoute = change_startCoil(coilRoute, costTable)
#	print "コスト",
#	print cost(costTable,coilRoute)
#	print coilRoute
    return coilRoute


def makeGraph(costTable, route, color):
    fig = plt.figure()
    ax = Axes3D(fig)
    print costTable
    print route
    k = 0
    for i in route:
        x = []
        y = []
        z = []
        for j in i:
            x.append(costTable[j][0])
            y.append(costTable[j][1])
            z.append(costTable[j][2])
        ax.plot(x, y, z, "o-", color="#" + color[k], ms=4, mew=0.5)
        k += 1
    ax.set_xlabel('board width')
    ax.set_ylabel('board thickness')
    ax.set_zlabel('coating thickness')
    plt.show()


def makeCSV(costTable, route):
    fig = plt.figure()
    ax = Axes3D(fig)
    for i in range(len(route)):
        plotRoute = route[i][0]
        x = []
        y = []
        z = []
        for j in plotRoute:
            costTable[i][0][j]


if __name__ == "__main__":
    sortNum = 1
    graphroute = []
    graphdata = []
    for i in range(1):
        filecoilRoute = "dataset90.csv"
        coilInfo = inputData(filecoilRoute)
        costTable = makeTable(coilInfo)
        minNnRoute = NN(costTable)
        min2optRoute = opt2(costTable, [j for j in range(len(coilInfo))])
        minNn2optRoute = opt2(costTable, minNnRoute)
        minSa2optRoute = sa2opt(costTable, [j for j in range(len(coilInfo))])
        minSaNn2optRoute = sa2opt(costTable, minNnRoute)
        route = [minNnRoute, min2optRoute, minNn2optRoute,
                 minSa2optRoute, minSaNn2optRoute]
        mincost = 1e51
        minroute = []
        for j in route:
            if mincost > cost(costTable, j):
                mincost = cost(costTable, j)
                minroute = j
        minroute = change_startCoil(minroute, costTable)

        for j in xrange(sortNum):
            f = open('sort' + str(i) + '.csv', 'w')
            writer = csv.writer(f, lineterminator='\n')
            for k in minroute:
                writer.writerow(coilInfo[j])
            f.close()
        graphroute.append(minroute[0:90])
        for j in range(3):
            graphroute.append(minroute[j * 30:(j + 1) * 30])
            graphdata.append(coilInfo)
    color = ["444444", "0000AA", "00AA00", "AA0000"]
    makeGraph(coilInfo, graphroute, color)

"""
	color=["0A0000"]
	makeGraph(coilInfo,[minSaNn2optRoute],color)
"""


In [0]:
#coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance as dis
from mpl_toolkits.mplot3d import Axes3D
import csv
from sklearn.cluster import KMeans

glapfcoilRoute=["sort0.csv","sort1.csv","sort2.csv"]

def inputData(filename):
	d = np.loadtxt(filename, delimiter=",")
	return d;
# 距離テーブルを作成

def makeSeikiTable():
	d=np.random.normal(0.5, 0.15, (90,3))
	size = len(d)
	table = np.ones((size, size))*-1
	# 幅:w 厚み:t 目付量:a
	dataMax = [1.0,1.0,1.0]
	dataMin = [0.0,0.0,0.0]
	# 制約条件 130%~70%まで許容
	# P1, Q1, R
	big = [0.3, 0.3, 0.3]
	# P2, Q2, R
	small = [0.3, 0.3, 0.3]
	costRate =[10.0,10.0,10.0]
	for i in xrange(size):
		for k in xrange(3):
			d[i][k]=(d[i][k]-dataMin[k])/(dataMax[k]-dataMin[k])
	for i in xrange(size):
		for j in xrange(size):
			if i==j:
				table[i][j] = 3e100
			else:
				diff = 0.0
				cost = 0.0
				for k in xrange(3):
					diff = abs(d[j][k]-d[i][k])
					if   d[i][k] > d[j][k] and diff > small[k]:
						cost = 1e50
						break
					elif d[i][k] < d[j][k] and diff > big[k]:
						cost = 1e50
						break
					else:
			                        funccost = abs(diff) *costRate[k]
						cost = cost + funccost
				table[i][j] = cost
	return table;

## 距離テーブルを作成
def makeTable(d):
	size = len(d)
	table = np.ones((size, size))*-1
	# 幅:w 厚み:t 目付量:a
	dataMax = [1800.0,3.2,0.040]
	dataMin = [600.0,0.4,0.007]
	# 制約条件 130%~70%まで許容
	# P1, Q1, R
	big = [0.5, 0.5, 0.5]
	# P2, Q2, R
	small = [0.5, 0.5, 0.5]
	costRate =[10.0,10.0,10.0]
	for i in xrange(size):
		for j in xrange(size):
			if i==j:
				table[i][j] = 3e100
			else:
				diff = 0.0
				cost = 0.0
				for k in xrange(3):
					diff = abs(d[j][k]-d[i][k])
					if   d[i][k] > d[j][k] and diff > small[k]:
						cost = 1e50
						break
					elif d[i][k] < d[j][k] and diff > big[k]:
						cost = 1e50
						break
					else:
			                        funccost = abs(diff) *costRate[k]
						cost = cost + funccost
				table[i][j] = cost

	return table;

def cost(costTable,route):
	cost=0
	for i in range(len(route)-1):
		cost+=costTable[route[i]][route[i+1]]
	return float(cost)

#一度数珠繋ぎにして最大のコストの部分を始点と終点となるようにする
#局所解の時、始点と終点のコイルが決まっていなければ解を有効
#(結局これに頼ってるようでは最適化できてないし、これで求まった解も局所解でしかない)
def change_startCoil(coilList,costTable):
	#print "before:",
	#print coilList

	route=[]
	maxcost=costTable[int(coilList[0])][int(coilList[1])]
	start=0
	for i in xrange(len(coilList)):
		next=int(coilList[i])
		prev=int(coilList[i-1])
		#print coilList
		if maxcost<costTable[prev][next]:
			maxcost=costTable[prev][next]
			start=i
	for i in xrange(start,len(coilList)):
		route.append(coilList[i])
	for i in xrange(0,start):
		route.append(coilList[i])
	#print "After:",
	#print route
	return route

#Nearest Neighbour法 (貪欲法)
# 次に近いものを今までのルートの中で一番コストが低くなる位置に再配置
def NN(costTable):
	coilNum=len(costTable)
	willCoil=[]
	for i in range(coilNum):
		willCoil.append(i)
	nowCoil=int(np.random.random()*coilNum%coilNum)
	willCoil.remove(nowCoil)
	nextCoil=0
	coilRoute=[]
	cost=0
	for i in range(coilNum-2):
		coilRoute.append(nowCoil)
		minCost=1e101
		for j in willCoil:
			if costTable[nowCoil][j]<minCost:
				nextCoil=j
				minCost=costTable[nowCoil][nextCoil]
		nowCoil=nextCoil
		cost+=minCost
		willCoil.remove(nextCoil)
	coilRoute.append(nowCoil)
	coilRoute.append(willCoil[0])
	coilRoute=change_startCoil(coilRoute,costTable)
	#print "cost:",
	#print cost
	#print coilRoute
	return coilRoute


#2opt()
def opt2(costTable, coilRoute):
	size=len(coilRoute)
	total = 0
	while True:
		count = 0
		#coilRoute=change_startCoil(coilRoute,costTable)
		for i in xrange(size - 2):
			i1 = i + 1
			for j in xrange(i + 2, size-1):
				if j == size - 1:
					j1 = 0
				else:
					j1 = j + 1
				if i != 0 or j1 != 0:
					l1 = costTable[coilRoute[i]][coilRoute[i1]]
					l2 = costTable[coilRoute[j]][coilRoute[j1]]
					l3 = costTable[coilRoute[i]][coilRoute[j]]
					l4 = costTable[coilRoute[i1]][coilRoute[j1]]
					if l1 + l2 > l3 + l4:
						new_coilRoute = coilRoute[i1:j+1]
						coilRoute[i1:j+1] = new_coilRoute[::-1]
						count += 1
		total += count
		if count == 0: break
#	print "コスト",
#	print cost(costTable,coilRoute)
#	print coilRoute
	return coilRoute

#2opt+焼きなまし
def sa2opt(costTable,coilRoute):
	temp=10
	size=len(coilRoute)
	total = 0
	while temp>1e-6:
		count = 0
		for i in xrange(size - 2):
			i1 = i + 1
			for j in xrange(i + 2, size):
				if j == size - 1:
					j1 = 0
				else:
					j1 = j + 1
				if i != 0 or j1 != 0:
					l1 = costTable[coilRoute[i]][coilRoute[i1]]
					l2 = costTable[coilRoute[j]][coilRoute[j1]]
					l3 = costTable[coilRoute[i]][coilRoute[j]]
					l4 = costTable[coilRoute[i1]][coilRoute[j1]]
					if l1 + l2 > l3 + l4:
						new_coilRoute = coilRoute[i1:j+1]
						coilRoute[i1:j+1] = new_coilRoute[::-1]
						count += 1
					else:
						dE=l1+l2-(l3+l4)
						temp*=0.99995
						if np.exp(dE/temp)>np.random.random():
							new_coilRoute = coilRoute[i1:j+1]
							coilRoute[i1:j+1] = new_coilRoute[::-1]
		total += count
		if count == 0: break
	coilRoute=change_startCoil(coilRoute,costTable)
#	print "コスト",
#	print cost(costTable,coilRoute)
#	print coilRoute
	return coilRoute



def makeGraph(costTable,route,color):
	fig = plt.figure()
	ax = Axes3D(fig)

	for i in range(len(route)):
		plotRoute=route[i][0]
		x=[]
		y=[]
		z=[]

		for j in plotRoute:
			x.append(costTable[i][0][j][0])
			y.append(costTable[i][0][j][1])
			z.append(costTable[i][0][j][2])
		ax.plot(x, y, z, "o-", color="#"+color[i], ms=4, mew=0.5)
	ax.set_xlabel('board width')
	ax.set_ylabel('board thickness')
	ax.set_zlabel('coating thickness')
	plt.show()

def makeCSV(costTable,route):
	fig = plt.figure()
	ax = Axes3D(fig)

	for i in range(len(route)):
		plotRoute=route[i][0]
		x=[]
		y=[]
		z=[]

		for j in plotRoute:
			costTable[i][0][j]



if __name__=="__main__":
  sortNum=3
  sort=[]
  for i in xrange(sortNum):
    sort.append([])
  #features = np.array(inputData("dataset90.csv"))
  d = inputData("dataset90.csv")
  dataMax = [1800.0,3.2,0.040]
  dataMin = [600.0,0.4,0.007]
  for i in xrange(len(d)):
  	for k in xrange(3):
  		d[i][k]=(d[i][k]-dataMin[k])/(dataMax[k]-dataMin[k])
  features=d
  kmeans_model = KMeans(n_clusters=3, random_state=10).fit(features)
  labels = kmeans_model.labels_
  for label, feature in zip(labels, features):
	  sort[int(label)].append(list(feature))
# ファイルオープン
  for i in xrange(sortNum):
    f = open('sort'+str(i)+'.csv', 'w')
    writer = csv.writer(f, lineterminator='\n')
    for j in sort[i]:
		writer.writerow(j)
    f.close()
  graphroute=[[] for  i in range(sortNum)]
  graphdata=[[] for  i in range(sortNum)]

  for i in range(sortNum):
	filecoilRoute="sort"+str(i)+".csv"
	coilInfo=inputData(filecoilRoute)
	print coilInfo
	costTable=makeTable(coilInfo)
	print costTable
	minNnRoute=NN(costTable)
	min2optRoute=opt2(costTable,[j for j in range(len(coilInfo))])
	minNn2optRoute=opt2(costTable,minNnRoute)
	minSa2optRoute=sa2opt(costTable,[j for j in range(len(coilInfo))])
	minSaNn2optRoute=sa2opt(costTable,minNnRoute)
	route=[minNnRoute,min2optRoute,minNn2optRoute,minSa2optRoute,minSaNn2optRoute]
	mincost=1e51
	minroute=[]
	for j in route:
		if mincost>cost(costTable,j):
			mincost=cost(costTable,j)
			minroute=j
	for j in xrange(sortNum):
		f = open('sort'+str(i)+'.csv', 'w')
		writer = csv.writer(f, lineterminator='\n')
		for k in minroute:
			writer.writerow(coilInfo[j])
		f.close()
	graphroute[i].append(minroute)
	graphdata[i].append(coilInfo)
  color=["0000AA","00AA00","AA0000"]
  makeGraph(graphdata,graphroute,color)

"""
	color=["0A0000"]
	makeGraph(coilInfo,[minSaNn2optRoute],color)
"""


In [0]:
#coding:utf-8
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.spatial import distance as dis
from mpl_toolkits.mplot3d import Axes3D


glapfcoilRoute=["sort0.csv","sort1.csv","sort2.csv"]

def inputData(filename):
	d = np.loadtxt(filename, delimiter=",")
	return d;
# 距離テーブルを作成
def makeSeikiTable():
	d=np.random.normal(0.5, 0.15, (30,3))
	size = len(d)
	table = np.ones((size, size))*-1
	# 幅:w 厚み:t 目付量:a
	dataMax = [1.0,1.0,1.0]
	dataMin = [0.0,0.0,0.0]
	# 制約条件 130%~70%まで許容
	# P1, Q1, R
	big = [0.5, 0.5, 0.5]
	# P2, Q2, R
	small = [0.5, 0.5, 0.5]
	costRate =[10.0,10.0,10.0]
	for i in xrange(size):
		for k in xrange(3):
			d[i][k]=(d[i][k]-dataMin[k])/(dataMax[k]-dataMin[k])
	for i in xrange(size):
		for j in xrange(size):
			if i==j:
				table[i][j] = 3e100
			else:
				diff = 0.0
				cost = 0.0
				for k in xrange(3):
					diff = abs(d[j][k]-d[i][k])
					if   d[i][k] > d[j][k] and diff > small[k]:
						cost = 1e50
						break
					elif d[i][k] < d[j][k] and diff > big[k]:
						cost = 1e50
						break
					else:
			                        funccost = abs(diff) *costRate[k]
						cost = cost + funccost
				table[i][j] = cost
	return table;

## 距離テーブルを作成
def makeTable(d):
	size = len(d)
	table = np.ones((size, size))*-1
	# 幅:w 厚み:t 目付量:a
	dataMax = [1800.0,3.2,40.0]
	dataMin = [600.0,0.4,7.0]
	big = [0.5, 0.5, 0.5]
	small = [0.5, 0.5, 0.5]
	costRate =[10.0,10.0,10.0]
	for i in xrange(size):
		for k in xrange(3):
			d[i][k]=(d[i][k]-dataMin[k])/(dataMax[k]-dataMin[k])
	for i in xrange(size):
		for j in xrange(size):
			if i==j:
				table[i][j] = 3e100
			else:
				diff = 0.0
				cost = 0.0
				for k in xrange(3):
					diff = abs(d[j][k]-d[i][k])
					if   d[i][k] > d[j][k] and diff > small[k]:
						cost = 1e50
						break
					elif d[i][k] < d[j][k] and diff > big[k]:
						cost = 1e50
						break
					else:
			                        funccost = abs(diff) *costRate[k]
						cost = cost + funccost
				table[i][j] = cost
	return table;

def cost(costTable,route):
	cost=0
	for i in range(len(route)-1):
		cost+=costTable[route[i]][route[i+1]]
	return float(cost)

#一度数珠繋ぎにして最大のコストの部分を始点と終点となるようにする
#局所解の時、始点と終点のコイルが決まっていなければ解を有効
#(結局これに頼ってるようでは最適化できてないし、これで求まった解も局所解でしかない)
def change_startCoil(coilList,costTable):
	#print "before:",
	#print coilList
	route=[]
	maxcost=costTable[int(coilList[-1])][int(coilList[0])]
	start=0
	for i in xrange(len(coilList)):
		next=int(coilList[i])
		prev=int(coilList[i-1])
		if maxcost<costTable[prev][next]:
			maxcost=costTable[prev][next]
			start=i
	#if start!=0:
		#print "change"
	for i in xrange(start,len(coilList)):
		route.append(coilList[i])
	for i in xrange(0,start):
		route.append(coilList[i])
	#print "After:",
	#print route
	return route

#Nearest Neighbour法 (貪欲法)
# 次に近いものを今までのルートの中で一番コストが低くなる位置に再配置
def NNMultiStart(costTable):
	coilNum=len(costTable)
	i=int(np.random.random()*coilNum%coilNum)
	Route=NN(costTable,i)
	minRoute=Route
	minCost=cost(costTable,minRoute)
	#print minCost
	for i in range(len(costTable)):
		route=NN(costTable,i)
		if minCost>cost(costTable,route):
			minRoute=route
			minCost=cost(costTable,route)
	#print minCost
	#print [len(Route),len(minRoute)]

	#print [cost(costTable,Route),cost(costTable,minRoute)]

	a=[Route,minRoute]
	#print a
	return a

def NN(costTable,start):
	coilNum=len(costTable)
	willCoil=[]
	for i in range(coilNum):
		willCoil.append(i)
	nowCoil=start
	willCoil.remove(nowCoil)
	nextCoil=0
	coilRoute=[]
	cost=0
	for i in range(coilNum-2):
		coilRoute.append(nowCoil)
		minCost=1e101
		for j in willCoil:
			if costTable[nowCoil][j]<minCost:
				nextCoil=j
				minCost=costTable[nowCoil][nextCoil]
		nowCoil=nextCoil
		willCoil.remove(nextCoil)
	coilRoute.append(nowCoil)
	coilRoute.append(willCoil[0])
	#coilRoute=change_startCoil(coilRoute,costTable)
	#print "cost:",
	#print coilRoute
	return coilRoute
#2opt()
def opt2(costTable, coilRoute):
	#coilRoute=change_startCoil(coilRoute,costTable)
	size=len(coilRoute)
	total = 0
	while True:
		count = 0
		#coilRoute=change_startCoil(coilRoute,costTable)
		for i in xrange(size - 2):
			i1 = i + 1
			for j in xrange(i + 2, size-1):
				j1 = j + 1
				l1 = costTable[coilRoute[i]][coilRoute[i1]]
				l2 = costTable[coilRoute[j]][coilRoute[j1]]
				l3 = costTable[coilRoute[i]][coilRoute[j]]
				l4 = costTable[coilRoute[i1]][coilRoute[j1]]
				if l1 + l2 > l3 + l4:
					new_coilRoute = coilRoute[i1:j1]
					coilRoute[i1:j1] = new_coilRoute[::-1]
					#print "swap"
					count += 1

		total += count
		if count == 0: break
	#print "コスト",
	#print cost(costTable,coilRoute),
	#print coilRoute
	coilRoute=change_startCoil(coilRoute,costTable)
	return coilRoute

#2opt+焼きなまし
def sa2opt(costTable,coilRoute):
	temp=20
	size=len(coilRoute)
	total = 0
	while temp>2e-5:
		count = 0
		for i in xrange(size - 2):
			i1 = i + 1
			for j in xrange(i + 2, size):
				if j == size - 1:
					j1 = 0
				else:
					j1 = j + 1
				if i != 0 or j1 != 0:
					l1 = costTable[coilRoute[i]][coilRoute[i1]]
					l2 = costTable[coilRoute[j]][coilRoute[j1]]
					l3 = costTable[coilRoute[i]][coilRoute[j]]
					l4 = costTable[coilRoute[i1]][coilRoute[j1]]
					if l1 + l2 > l3 + l4:
						new_coilRoute = coilRoute[i1:j+1]
						coilRoute[i1:j+1] = new_coilRoute[::-1]
						count += 1
					else:
						dE=l1+l2-(l3+l4)
						temp*=0.99995
						#print dE,temp,np.exp(dE/temp)
						if np.exp(dE/temp)>np.random.random():
							new_coilRoute = coilRoute[i1:j+1]
							coilRoute[i1:j+1] = new_coilRoute[::-1]
							count += 1
		total += count
		if count == 0: break
	coilRoute=change_startCoil(coilRoute,costTable)
	#print "コスト",
	#print cost(costTable,coilRoute),
	#print coilRoute
	return coilRoute



def makeGraph(costTable,route,color):
	fig = plt.figure()
	ax = Axes3D(fig)

	for i in range(len(route)):
		plotRoute=route[i]
		x=[]
		y=[]
		z=[]
		for j in plotRoute:
			x.append(costTable[j][0])
			y.append(costTable[j][1])
			z.append(costTable[j][2])
		ax.plot(x, y, z, "o-", color="#"+color[i], ms=4, mew=0.5)
	ax.set_xlabel('board width')
	ax.set_ylabel('board thickness')
	ax.set_zlabel('coating thickness')
	plt.show()


if __name__=="__main__":
	costGraph=[[],[],[],[],[],[],[],[]]
#	for randomSeed in range(100):
	minRate=[0 for i in xrange(8)]
	for trialNo in range(1000):
			#filecoilRoute="syntanic_data_normal1.csv"
			#coilInfo=inputData(filecoilRoute)
			costTable=makeSeikiTable()
			#print "\nNN法"
			#minNnRoute=NN(costTable,0)
			#print "\nNNマルチ"
			minNnMulti=NNMultiStart(costTable)
			#print "\n2-opt単体"
			costGraph[0].append(cost(costTable,minNnMulti[0]))
			costGraph[1].append(cost(costTable,minNnMulti[1]))

			min2optRoute=opt2(costTable,[i for i in range(len(costTable))])
		#	print "\nNN+2-opt単体"
			costGraph[2].append(cost(costTable,min2optRoute))

			minNn2optRoute=opt2(costTable,minNnMulti[0])
			costGraph[3].append(cost(costTable,minNn2optRoute))
			minNn2optRoute=opt2(costTable,minNnMulti[1])
			costGraph[4].append(cost(costTable,minNn2optRoute))
		#	minNn2optRoute=opt2(costTable,minNnMulti[1])
		#	print "\n焼きなまし法を用いた2-opt単体"
			minSa2optRoute=sa2opt(costTable,[i for i in range(len(costTable))])
			costGraph[5].append(cost(costTable,minSa2optRoute))

		#	print "\n焼きなまし法を用いたNN+2-opt"
			minSaNn2optRoute=sa2opt(costTable,minNnMulti[0])
			costGraph[6].append(cost(costTable,minSaNn2optRoute))


			costGraph[7].append(cost(costTable,sa2opt(costTable,minNnMulti[1])))



			mincost=1e100
			for i in xrange(len(costGraph)):
				if mincost>costGraph[i][-1]:
					mincost=costGraph[i][-1]
					bestWay=i
			minRate[bestWay]+=1

			"""
			#SminSa2optRoute=sa2Opt(costTable,minNnRoute)
			print minNnMulti
			routes=[minNnMulti[0],minNnMulti[1],min2optRoute,minNn2optRoute,minSa2optRoute,minSaNn2optRoute]
			print routes
			i=0
			for route in routes:
				print str(cost(costTable,route))+",",
				costGraph[i].append(cost(costTable,route))
				i+=1
			print ""
			"""
	data = (costGraph[0],costGraph[1],costGraph[2],costGraph[3],costGraph[4],costGraph[5],costGraph[6],costGraph[7])
	fig = plt.figure()
	ax = fig.add_subplot(111)
		# 箱ひげ図をつくる
	bp = ax.boxplot(data)
	for rate in minRate:
		print rate/1000.0,
	xticks = ['NN', 'NN\n(min)', '2opt\n(random)','2opt\n(NN(random))','2opt\n(NN(min))','SA2opt\n(random)','SA2opt\n(NN(random))','SA2opt\n(NN(min))',  ]
	plt.xticks([1,2,3,4,5,6,7,8], xticks)
	plt.ylim([0,100])
	plt.show()


0.0 0.003 0.005 0.02 0.125 0.333 0.275 0.239



#	color=["0A0000"]
	#makeGraph(coilInfo,[minSaNn2optRoute],color)
